In [1]:
import os, sys
import numpy as np
import pandas as pd

from config_local import local_config

In [2]:

from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.model_selection import KFold, RandomizedSearchCV

In [3]:
train = pd.read_csv(local_config.TRAIN_PROCESS6_CSV)
test  = pd.read_csv(local_config.TEST_PROCESS6_CSV)
testRaw = pd.read_csv(local_config.TEST_CSV, index_col="Id")

In [4]:
y = train["logSP"]                 # your target column (log SalePrice)
X = train.drop(columns=["logSP"])  # all features except target

In [5]:
# ==== Cross-validation + Random Search ====
kf = KFold(n_splits=5, shuffle=True, random_state=42)

xgb = XGBRegressor(
    objective="reg:squarederror",
    random_state=42,
    n_jobs=-1,
    eval_metric="rmse",
    tree_method="hist",   # <- histogram method
    device="cuda"         # <- run it on GPU
)

param_dist = {
    "n_estimators": [800, 1000, 1200],
    "learning_rate": [0.05, 0.04, 0.03],
    "max_depth": [3, 4, 5],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.7, 0.9],
}

random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=30,                  # a bit more combos
    scoring="neg_mean_squared_error",
    cv=5,                       # more reliable CV
    n_jobs=-1,
    verbose=2,
    random_state=42,
)

print("Running RandomizedSearchCV for XGBoost...")
random_search.fit(X.values, y.values)   # <-- use .values to avoid feature name issues

print("\nBest params found:")
print(random_search.best_params_)

best_mse = -random_search.best_score_
best_rmse = best_mse ** 0.5
print(f"\nBest CV RMSE from RandomizedSearch: {best_rmse:.4f}")

Running RandomizedSearchCV for XGBoost...
Fitting 5 folds for each of 30 candidates, totalling 150 fits

Best params found:
{'subsample': 0.8, 'n_estimators': 1200, 'max_depth': 3, 'learning_rate': 0.04, 'colsample_bytree': 0.9}

Best CV RMSE from RandomizedSearch: 0.1148


In [6]:
# ==== Final model (already refit on all data) ====
best_model = random_search.best_estimator_

# ==== Predictions on test ====
test_pred_log = best_model.predict(test)

# Inverse log transform (same as LightGBM)
test_pred_real = np.expm1(test_pred_log)
# or, if you used log1p: test_pred_real = np.expm1(test_pred_log)

# ==== Submission (same style as ElasticNet / LightGBM) ====
submission = pd.DataFrame({
    "Id": testRaw.index,
    "SalePrice": test_pred_real
})

out_path = os.path.join(local_config.SUBMISSIONS_DIR, "xgboost_Model.csv")
submission.to_csv(out_path, index=False)

print(f"Submission saved: {out_path}")

Submission saved: D:\Project\Kaggle\house-prices-starter\data\submissions\xgboost_Model.csv


d:\Project\Kaggle\house-prices-starter\.venv\Lib\site-packages\xgboost\core.py:774: UserWarning: [19:29:20] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:62: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)
